In [3]:
import datetime, time
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from bs4 import BeautifulSoup
import requests

In [4]:
csv = pd.read_csv('./data/3-15_22.csv')

In [12]:
csv.loc[:1, ['goal', 'usd_pledged']]

,goal,usd_pledged
0,6666,7916.0
1,750,1626.0


In [19]:
csv['percentage'] = csv['usd_pledged'] / csv['goal']
csv['duration'] = csv['deadline'] - csv['launched_at']


In [23]:
def boolean_to_number(x):
    if x is True:
        return 1
    elif x is False:
        return 0

csv['spotlight_num'] = csv['spotlight'].apply(boolean_to_number)

In [32]:
target_features = ['usd_pledged', 'backers_count', 'percentage']
input_features = ['goal', 'deadline', 'launched_at', 'duration', 'spotlight_num'] + list(csv.columns[33:-3])

In [48]:
len(input_features)

44930

In [50]:
from sklearn import linear_model
from sklearn import preprocessing

x = csv[input_features] # 44930 (5 + 44925)
y = csv[target_features] # 3

print ("... Normalize features")
min_max_scalar = preprocessing.MinMaxScaler()
x_scaled = min_max_scalar.fit_transform(x)
x_scaled = pd.DataFrame(x_scaled)
y_scaled = min_max_scalar.fit_transform(y)
y_scaled = pd.DataFrame(y_scaled)

print ("... Training")
reg = linear_model.Ridge(alpha=.5)
reg.fit(x_scaled, y_scaled)

... Normalize features
... Training


Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [57]:
coef = pd.DataFrame(reg.coef_, columns=input_features)

In [58]:
coef

,goal,deadline,launched_at,duration,spotlight_num,0,1,2,3,4,...,44915,44916,44917,44918,44919,44920,44921,44922,44923,44924
0,-0.000060,-0.001206,-0.001221,0.000147,0.000998,-0.000385,0.000050,0.000281,-0.000273,-0.000088,...,-0.000043,-0.000043,-0.000043,-0.000043,0.000003,0.0,0.0,0.0,0.0,0.0
1,-0.000055,-0.001075,-0.001086,0.000051,0.001591,-0.000433,0.000066,0.000293,-0.000283,0.000031,...,-0.000040,-0.000040,-0.000040,-0.000040,0.000003,0.0,0.0,0.0,0.0,0.0
2,0.000076,0.000027,0.000016,0.000324,0.000887,-0.000826,-0.000034,-0.000304,0.000043,0.000450,...,0.000100,0.000100,0.000100,0.000100,0.000121,0.0,0.0,0.0,0.0,0.0


In [71]:
target_idx = 1
mx = max(reg.coef_[target_idx])
for idx in range(len(reg.coef_[target_idx])):
    if reg.coef_[target_idx][idx] == mx:
        print (idx)

6480
